[ RNN 모델 ]
- 순환 신경망(RNN) : 각 시점의 데이터가 이전 시점 데이터와 독립적이지 않음 
- 연속형 데이터(자연어도 속함)를 입력받아, 각 시점 마다 "Hidden State" 형태로 저장 : 의미와 문맥을 동시에 학습
- 문제점 : 장기 종속성 문제(입력 데이터가 길어지면 과거 정보를 잊어버림)

In [72]:
from torch import nn

In [73]:
# 문장 분류 모델 클래스 생성

class SentenceClassifier(nn.Module):
    
    def __init__(self, 
                 n_vocab,             # n_vocab: 단어 사전(vocabulary)의 크기 (총 단어 수)
                 hidden_dim,          # hidden_dim: RNN의 은닉 상태(hidden state)의 크기 (출력 차원)
                 embedding_dim,       # embedding_dim: 단어 임베딩(embedding) 벡터의 차원 (특징 수)
                 n_layers,            # n_layers: RNN 층의 개수 (몇 층으로 쌓을지 결정)
                 dropout=0.5,         # dropout: 드롭아웃(dropout) 비율 (과적합 방지를 위한 노드 무작위 비활성화 비율)
                 bidirectional=True,  # bidirectional: 양방향 RNN 여부 (True: 양방향, False: 단방향)
                 model_type="rnn"):
        super().__init__()
        
        
        # 임베딩
        self.embedding = nn.Embedding(num_embeddings=n_vocab,
                                      embedding_dim=embedding_dim,
                                      padding_idx=0)
        
        
        if model_type == 'rnn':
            self.model = nn.RNN(input_size=embedding_dim,
                                hidden_size=hidden_dim,
                                num_layers=n_layers,
                                bidirectional=bidirectional,
                                dropout=dropout,
                                batch_first=True)
            
            
        elif model_type == 'lstm':
            self.model = nn.LSTM(input_size=embedding_dim,
                                hidden_size=hidden_dim,
                                num_layers=n_layers,
                                bidirectional=bidirectional,
                                dropout=dropout,
                                batch_first=True)
        
        
        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2, 1)
        else:
            self.classifier = nn.Linear(hidden_dim,1)
        
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings)
        last_output = output[:,-1,:]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

In [74]:
# 파일 불러오기
import pandas as pd


text = 'IMDb_Reviews.csv'

corpus_df = pd.read_csv(text)

In [75]:
corpus_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [76]:
# 데이터의 균형성 확인
corpus_df['sentiment'].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [77]:
# train, test 분리
train = corpus_df.sample(frac=0.9, random_state=42)
test = corpus_df.drop(train.index)

In [78]:

# 데이터 토큰화 및 단어 사전 구축
from konlpy.tag import Okt
from collections import Counter

def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter()
    
    for tokens in corpus:
        counter.update(tokens)
        
    vocab = special_tokens
    
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
        
    return vocab

In [79]:
tokenizer = Okt()
train_tokens = [tokenizer.morphs(review) for review in train.review]
test_tokens = [tokenizer.morphs(review) for review in test.review]

vocab = build_vocab(corpus=train_tokens, n_vocab=5000, special_tokens=["<pad>","<unk>"])
token_to_id = {token:idx for idx,token in enumerate(vocab)}
id_to_token = {idx:token for idx,token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))  # 스페셜 토큰 + 5000개의 단어 사전 크기

['<pad>', '<unk>', 'the', ',', '.', 'and', 'a', 'of', 'to', "'"]
5002


In [80]:
# 정수 인코딩 및 패딩
import numpy as np

def pad_sequences(sequences, max_length, pad_value):
    result = list()
    
    for sequence in sequences:
        sequence = sequence[:max_length]
        pad_length = max_length - len(sequence)
        padded_sequence = sequence + [pad_value] * pad_length
        result.append(padded_sequence)
        
    return np.asarray(result)

In [81]:
unk_id = token_to_id["<unk>"]

train_ids = [[token_to_id.get(token,unk_id) for token in review] for review in train_tokens]
test_ids = [[token_to_id.get(token,unk_id) for token in review] for review in test_tokens]

In [82]:
max_length = 32
pad_id = token_to_id["<pad>"]

train_ids = pad_sequences(train_ids, max_length, pad_id)
test_ids = pad_sequences(test_ids, max_length, pad_id)

print(train_ids[0])
print(test_ids[0])

[  13  514    8  141   17   28   51 1012  106   13   74  140    4  119
   14   22    6  566 1499   29   11   19   11   21   13  376 2091    1
    9   16  715   12]
[ 137   10   82  855   15 2378   10   43  263   40    7    2  148  124
  149   36   76   12  117  561   30  110 1102    4   13  141    2  124
 1054  915   49    2]


In [83]:
# 데이터로더 적용 
import torch
from torch.utils.data import TensorDataset, DataLoader

train_ids = torch.tensor(train_ids)
test_ids = torch.tensor(test_ids)

train_labels = torch.tensor(train.sentiment.values, dtype=torch.float32)
test_labels = torch.tensor(test.sentiment.values, dtype=torch.float32)

# 데이터 셋 생성
train_dataset = TensorDataset(train_ids, train_labels)
test_dataset = TensorDataset(test_ids, test_labels)

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [84]:
# 손실 함수와 최적화 함수 정의
from torch import optim

n_vocab = len(token_to_id)
hidden_dim = 64
embedding_dim = 128
n_layers = 2

In [85]:
device = "cuda" if torch.cuda.is_available() else "cpu"
classifier = SentenceClassifier(n_vocab=n_vocab,
                                hidden_dim=hidden_dim,
                                embedding_dim=embedding_dim,
                                n_layers=n_layers).to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)

In [86]:
# 모델 학습 및 테스트
def train(model, datasets, criterion, optimizer, device, interval):
    model.train()
    losses = list()
    
    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)
        
        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step % interval == 0:
            print(f"Train Loss {step} : {np.mean(losses)}")

In [87]:
def test(model, datasets, criterion, device):
    model.eval()
    losses = list()
    corrects = list()
    
    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)
        
        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())
        yhat = torch.sigmoid(logits) > .5
        corrects.extend(torch.eq(yhat, labels).cpu().tolist())
        
    print(f"Val Loss : {np.mean(losses)}, Val Accuracy {np.mean(corrects)}")

In [88]:
epochs = 5
interval = 500

for epoch in range(epochs):
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device)

Train Loss 0 : 0.6878344416618347
Train Loss 500 : 0.7076761421567189
Train Loss 1000 : 0.7029587475808112
Train Loss 1500 : 0.6981316920441838
Train Loss 2000 : 0.6947866791430144
Train Loss 2500 : 0.6922980737133247
Val Loss : 0.6922418543705925, Val Accuracy 0.5224
Train Loss 0 : 0.7085254192352295
Train Loss 500 : 0.6968713719211891
Train Loss 1000 : 0.695765037755747
Train Loss 1500 : 0.6949305514507814
Train Loss 2000 : 0.6925833672359548
Train Loss 2500 : 0.6907259718697818
Val Loss : 0.6708704319815285, Val Accuracy 0.5972
Train Loss 0 : 0.6958054304122925
Train Loss 500 : 0.6752775162160991
Train Loss 1000 : 0.6740325580050538
Train Loss 1500 : 0.6726337632959799
Train Loss 2000 : 0.6720188366270613
Train Loss 2500 : 0.670213126578554
Val Loss : 0.6591296685389436, Val Accuracy 0.607
Train Loss 0 : 0.4807392954826355
Train Loss 500 : 0.6519864321111919
Train Loss 1000 : 0.6480890119766498
Train Loss 1500 : 0.6487406439816134
Train Loss 2000 : 0.6474773132282755
Train Loss 2500

In [89]:
# 학습된 모델로부터 임베딩 추출
token_to_embedding = dict()
embedding_matrix = classifier.embedding.weight.detach().cpu().numpy()

for word, emb in zip(vocab, embedding_matrix):
    token_to_embedding[word] = emb
    
token = vocab[1000]
print(token, token_to_embedding[token])

setting [-0.25182456  0.26007736 -0.620779    1.4288586   0.6850555  -0.14038883
 -1.8222723   1.4691087   0.14624998 -0.6025309   1.7174916  -2.2324169
  0.389503   -0.1725249   0.12300435  0.9271532   0.39208105  1.0558177
  0.86882764 -0.672931    1.7487837   0.98854655 -0.05701118  0.6886515
 -1.5732802  -0.5079216   0.40737304  1.4098773  -0.9733842   0.8676472
 -0.34807047  0.89049155 -1.3570387   0.36670005 -1.0284475  -1.6622175
 -0.6718964   0.66750735 -0.5785393   1.833308    2.379144    0.37998292
 -1.1240984  -1.8942959  -1.298464   -1.9551221   0.9119829  -2.0350928
  1.5303962   0.09471378  0.21837656 -1.8628975  -0.7058022   1.4744684
  0.12435835  0.7056739  -0.8061879  -0.49389538 -0.78420985  1.1959054
 -0.378505   -0.2638521   0.6341217  -0.64404964 -0.33460405 -0.3618548
  0.20871876  1.0132858   0.12031981 -0.04590784 -0.4947955   0.4707805
  2.40968     1.0931921  -0.48616046 -0.56204885 -0.95470864  0.8112035
  0.33667094  1.0611689  -0.27316606  0.15434018  1.69

In [90]:
from gensim.models import Word2Vec
import gensim

In [91]:
# 모델 학습
word2vec = Word2Vec(sentences=train_tokens,
                    vector_size=128,
                    window=5,
                    min_count=1,
                    sg=1,
                    epochs=3,
                    max_final_vocab=10000)

# 모델 저장
word2vec.save('./models/word2vec.model')

In [92]:
# 사전 학습된 모델로 임베딩 계층 초기화
from gensim.models import Word2Vec

word2vec = Word2Vec.load('./models/word2vec.model')
init_embeddings = np.zeros((n_vocab, embedding_dim))

for index,token in id_to_token.items(): 
    if token not in ["<pad>","<unk>"]: 
        init_embeddings[index] = word2vec.wv[token]

embedding_layer = nn.Embedding.from_pretrained(torch.tensor(init_embeddings, dtype=torch.float32))

In [93]:
# 사전 학습된 임베딩 계층 적용하도록 수정된 클래스
class SentenceClassifier(nn.Module):
    
    def __init__(self, 
                 n_vocab,             # n_vocab: 단어 사전(vocabulary)의 크기 (총 단어 수)
                 hidden_dim,          # hidden_dim: RNN의 은닉 상태(hidden state)의 크기 (출력 차원)
                 embedding_dim,       # embedding_dim: 단어 임베딩(embedding) 벡터의 차원 (특징 수)
                 n_layers,            # n_layers: RNN 층의 개수 (몇 층으로 쌓을지 결정)
                 dropout=0.5,         # dropout: 드롭아웃(dropout) 비율 (과적합 방지를 위한 노드 무작위 비활성화 비율)
                 bidirectional=True,  # bidirectional: 양방향 RNN 여부 (True: 양방향, False: 단방향)
                 pretrained_embedding=None, # 사전 학습된 임베딩 모델을 사용할 건지 여부
                 model_type="rnn"):
        super().__init__()
        
        
        # 사전 학습 임베딩 여부에 따라....
        if pretrained_embedding is not None:
            self.embedding = nn.Embedding.from_pretrained(torch.tensor(pretrained_embedding, dtype=torch.float32))
            
        else:
            self.embedding = nn.Embedding(num_embedding=n_vocab,
                                          embedding_dim=embedding_dim,
                                          padding_idx=0)
        
        
        # 모델 타입에 따라....
        if model_type == 'rnn':
            self.model = nn.RNN(input_size=embedding_dim,
                                hidden_size=hidden_dim,
                                num_layers=n_layers,
                                bidirectional=bidirectional,
                                dropout=dropout,
                                batch_first=True)
            
            
        elif model_type == 'lstm':
            self.model = nn.LSTM(input_size=embedding_dim,
                                hidden_size=hidden_dim,
                                num_layers=n_layers,
                                bidirectional=bidirectional,
                                dropout=dropout,
                                batch_first=True)
        
        
        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2, 1)
        else:
            self.classifier = nn.Linear(hidden_dim,1)
        
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings)
        last_output = output[:,-1,:]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

In [94]:
# 사전 학습된 임베딩을 사용한 모델 학습
classifier = SentenceClassifier(n_vocab=n_vocab,
                                hidden_dim=hidden_dim,
                                embedding_dim=embedding_dim,
                                n_layers=n_layers,
                                pretrained_embedding=init_embeddings).to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.RMSprop(classifier.parameters(),lr=0.001)
epochs = 5
interval = 500

for epoch in range(epochs):
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device)

Train Loss 0 : 0.6883194446563721
Train Loss 500 : 0.6931819531494035
Train Loss 1000 : 0.678467812148722
Train Loss 1500 : 0.6822168226443792
Train Loss 2000 : 0.6794830105383357
Train Loss 2500 : 0.679866569714754
Val Loss : 0.6928011893083493, Val Accuracy 0.5312
Train Loss 0 : 0.6952505111694336
Train Loss 500 : 0.682873096056803
Train Loss 1000 : 0.6775231925281254
Train Loss 1500 : 0.6828614726295319
Train Loss 2000 : 0.6832333537473017
Train Loss 2500 : 0.6835195777750835
Val Loss : 0.679693478364914, Val Accuracy 0.5676
Train Loss 0 : 0.7567013502120972
Train Loss 500 : 0.6751869537397297
Train Loss 1000 : 0.6725357744124505
Train Loss 1500 : 0.6685660567822097
Train Loss 2000 : 0.6675118626444891
Train Loss 2500 : 0.6692913259472288
Val Loss : 0.6618804388914626, Val Accuracy 0.6096
Train Loss 0 : 0.6960300207138062
Train Loss 500 : 0.6556171270425686
Train Loss 1000 : 0.6488232035915573
Train Loss 1500 : 0.6541718798307956
Train Loss 2000 : 0.6514954228570377
Train Loss 2500 